In [122]:
import re

gpc_name = [
  'Residential: Fuel Combustion',
  'Residential: Grid Energy',
  'Commercial & Institutional: Fuel Combustion',
  'Commercial & Institutional: Grid Energy',
  'Manufacturing: Fuel Combustion',
  'Manufacturing: Grid Energy',
  'Energy: Fuel Combustion',
  'Energy: Grid Energy',
  'Ag, Forestry & Fishing: Fuel Combustion',
  'Ag, Forestry & Fishing: Grid Energy',
  'Non-Specified: Fuel Combustion',
  'Non-Specified: Grid Energy',
  'Fugitive Emissions: Coal',
  'Fugitive Emissions: Oil & Natural Gas',
  'On-Road Transport: Fuel Combustion',
  'On-Road Transport: Grid Energy',
  'Railways: Fuel Combustion',
  'Railways: Grid Energy',
  'Waterborne: Fuel Combustion',
  'Waterborne: Grid Energy',
  'Aviation: Fuel Combustion',
  'Aviation: Grid Energy',
  'Off-Road Transport: Fuel Combustion',
  'Off-Road Transport: Grid Energy',
  'Solid Waste: Disposed Within City',
  'Solid Waste: Disposed Outside City',
  'Biological Waste: Treated Within City',
  'Biological Waste: Treated Outside City',
  'Incineration: Treated Within City',
  'Incineration: Treated Outside City',
  'Wastewater: Treated Within City',
  'Wastewater: Treated Outside City',
]
gpc_sector = [
  'I.1.1',
  'I.1.2',
  'I.2.1',
  'I.2.2',
  'I.3.1',
  'I.3.2',
  'I.4.1',
  'I.4.2',
  'I.5.1',
  'I.5.2',
  'I.6.1',
  'I.6.2',
  'I.7.1',
  'I.8.1',
  'II.1.1',
  'II.1.2',
  'II.2.1',
  'II.2.2',
  'II.3.1',
  'II.3.2',
  'II.4.1',
  'II.4.2',
  'II.5.1',
  'II.5.2',
  'III.1.1',
  'III.1.2',
  'III.2.1',
  'III.2.2',
  'III.3.1',
  'III.3.2',
  'III.4.1',
  'III.4.2',
]
gpc_scope = [
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  1,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  3,
]
gpc_3rd_level = [
  'residential_fuel',
  'residential_grid',
  'commercial_fuel',
  'commercial_grid',
  'manufacturing_fuel',
  'manufacturing_grid',
  'energy_fuel',
  'energy_grid',
  'agriculture_fuel',
  'agriculture_grid',
  'nonspecified_fuel',
  'nonspecified_grid',
  'fugitive_coal',
  'fugitive_oil',
  'onroad_fuel',
  'onroad_grid',
  'railways_fuel',
  'railways_grid',
  'waterborne_fuel',
  'waterborne_grid',
  'aviation_fuel',
  'aviation_grid',
  'offroad_fuel',
  'offroad_grid',
  'solidwaste_within',
  'solidwaste_outside',
  'biowaste_within',
  'biowaste_outside',
  'incineration_within',
  'incineration_outside',
  'wastewater_within',
  'wastewater_outside',
]
gpc_2nd_level = [
  'residential',
  'commercial',
  'manufacturing',
  'energy',
  'agriculture',
  'nonspecified',
  'fugitive',
  'onroad',
  'railways',
  'waterborne',
  'aviation',
  'offroad',
  'solidwaste',
  'biowaste',
  'incineration',
  'wastewater',
]
gpc_1st_level = ['building'] * 6 + ['other_stationary'] * 8 + ['transport'] * 10 + ['waste'] * 8
activities = [n + "_activity" for n in gpc_1st_level]
emission_factors = [n + "_emission_factor" for n in gpc_1st_level]
emissions = [n + "_emissions" for n in gpc_1st_level]
costs = [n + "_emission_costs" for n in gpc_1st_level]

In [162]:
nodes = []
nodes = nodes + [
    "################### GPC Framework\n\n" +
    "emission_sectors:"]

nodes = nodes + ["\n################## Emissions 2nd level"]
for i in range(len(gpc_2nd_level)):
    nodes = nodes + [
        "- id: " + gpc_2nd_level[i] + "_emissions\n  name: '" +
        re.sub('\\..?$', '', gpc_sector[i * 2]) + " " +
        re.sub(':|Fuel Combustion|Grid Energy|Coal|Disposed |Within |City|Treated |Outside ', '', gpc_name[i * 2]) +
        "Emissions'\n" +
        "  part_of: " + emissions[i * 2]
#        "  type: simple.AdditiveNode\n  quantity: emissions\n  unit: kt/a\n" +
#        "  output_nodes: [" + emissions[i * 2] + ", " + costs[i * 2] + "]"
    ]

nodes = nodes + ["\nnodes:\n"]
part1 = "  type: simple.AdditiveNode\n  quantity: emissions\n  unit: kt/a\n"
part2 = "  output_nodes:\n  - id: net_emissions\n  - id: "
part3 = "_emission_costs\n    to_dimensions:\n    - id: account\n      categories: [payments]\n"

nodes = nodes + [
    "################### Emissions 1st level\n" +
    "- id: net_emissions\n  name: Net emissions\n  is_outcome: true\n" + part1 +
    "- id: building_emissions\n  name: I Building Emissions\n  color: '#ff4f57'\n" +
    part1 + part2 + "building" + part3 +
    "- id: other_stationary_emissions\n  name: I Other Stationary Emissions\n  color: '#ff5f67'\n" +
    part1 + part2 + "other_stationary" + part3 +
    "- id: transport_emissions\n  name: II Transport Emissions\n  color: '#0047b6'\n" +
    part1 + part2 + "transport" + part3 +
    "- id: waste_emissions\n  name: III Waste Emissions\n  color: '#014B30'\n" +
    part1 + part2 + "waste" + part3
]

nodes = nodes + ["################## Emissions 3rd level"]
for i in range(len(gpc_scope)):
    nodes = nodes + [
        "- id: " + gpc_3rd_level[i] + "\n  name: '" + gpc_sector[i] + " " + gpc_name[i] + " Emissions'\n" + 
        "  type: simple.MultiplicativeNode\n  quantity: emissions\n  unit: kt/a\n" +
        "  input_nodes: [" + activities[i] + ", " + emission_factors[i] +
        "]\n  output_nodes: [" + [n + "_emissions" for n in gpc_2nd_level][int(i / 2)] + "]"
    ]

nodes = nodes + ["\n################## Activity nodes\n"]
for i in sorted(list(set(gpc_1st_level) - {'waste'})):
    nodes = nodes + [
        "- id: " + i + "_activity" + "\n  name: " + i + " activity\n" +
        "  type: simple.AdditiveNode\n  quantity: energy\n" +
        "  unit: GWh/a\n  historical_values: [[2010, 1]]\n  output_nodes: [total_energy_consumption]"
    ]
nodes = nodes + [
    "\n- id: waste_activity\n  name: Waste activity\n  type: simple.AdditiveNode\n" +
    "  quantity: mass\n  unit: m**3/a\n  historical_values: [[2010, 1]]\n\n" +
    "- id: total_energy_consumption\n  name: Total energy consumption\n  type: simple.AdditiveNode\n" +
    "  quantity: energy\n  unit: GWh/a\n  is_outcome: true"
]

nodes = nodes + ["\n################## Emission factor nodes\n"]
units = ["g/kWh"] * 4 + ["g/m**3"]
ids = sorted(list(set(gpc_1st_level + ['electricity'])))
for i in range(len(ids)):
    nodes = nodes + [
        "- id: " + ids[i] + "_emission_factor" + "\n  name: " + ids[i] + " emission factor\n" +
        "  type: simple.AdditiveNode\n  quantity: emission_factor\n" +
        "  unit: " + units[i] + "\n  historical_values: [[2010, 1]]"
    ]

nodes = nodes + ["\n################## Cost nodes\n"]
for i in sorted(list(set(gpc_1st_level))):
    nodes = nodes + [
        "- id: " + i + "_action_costs" + "\n  name: " + i + " action costs\n" +
        "  type: simple.AdditiveNode\n  quantity: currency\n  unit: kEUR/a\n" +
        "  output_nodes: [" + i + "_costs]\n" +
        "  input_datasets:\n  - id: budget/action_costs\n    forecast_from: 2024\n" +
        "  input_dimensions: [account]\n" +
        "  output_dimensions: [account]\n" +
        "- id: " + i + "_emission_costs" + "\n  name: " + i + " emission costs\n" +
        "  type: simple.MultiplicativeNode\n  quantity: currency\n  unit: kEUR/a\n" +
        "  input_nodes:\n  - id: price_of_emissions\n    to_dimensions:\n    - id: account\n    - categories: [payments]\n" +
        "  output_nodes: [" + i + "_costs]\n" +
        "  input_dimensions: [account]\n" +
        "  output_dimensions: [account]\n" +
        "- id: " + i + "_costs" + "\n  name: " + i + " costs\n" +
        "  type: simple.AdditiveNode\n  quantity: currency\n  unit: kEUR/a\n" +
        "  output_nodes: [total_monetary_benefits]\n" +
        "  input_dimensions: [account]\n" +
        "  output_dimensions: [account]"
    ]

nodes = nodes + [
    "- id: price_of_emissions\n  name: Price of emissions\n" +
    "  type: simple.AdditiveNode\n  quantity: unit_price\n  unit: EUR/t\n" +
    "  historical_values: [[2010, 100]]\n"
]

nodes = nodes + ["############### Collection nodes\n"]
for i in sorted(list(set(gpc_1st_level))):
    nodes = nodes + [
    "- id: collect_" + i + "\n  name: Collect " + i + " nodes\n" +
    "  description: This node does not contain meaningful content, but it can be used to filter nodes from the full graph.\n" +
    "  type: values.AssociationNode\n  quantity: number\n  unit: pcs\n  historical_values: [[2010, 0]]\n" +
    "  input_nodes:\n  - id: " + i + "_emissions\n  - id: " + i + "_costs\n" +
    "    from_dimensions:\n    - id: account\n      flatten: true"
    ]

f = open("nodes.txt", "w")
for node in nodes:
    print(node)
    f.write(node + "\n")
f.close()


################### GPC Framework

emission_sectors:

################## Emissions 2nd level
- id: residential_emissions
  name: 'I.1 Residential Emissions'
  part_of: building_emissions
- id: commercial_emissions
  name: 'I.2 Commercial & Institutional Emissions'
  part_of: building_emissions
- id: manufacturing_emissions
  name: 'I.3 Manufacturing Emissions'
  part_of: building_emissions
- id: energy_emissions
  name: 'I.4 Energy Emissions'
  part_of: other_stationary_emissions
- id: agriculture_emissions
  name: 'I.5 Ag, Forestry & Fishing Emissions'
  part_of: other_stationary_emissions
- id: nonspecified_emissions
  name: 'I.6 Non-Specified Emissions'
  part_of: other_stationary_emissions
- id: fugitive_emissions
  name: 'I.7 Fugitive Emissions Emissions'
  part_of: other_stationary_emissions
- id: onroad_emissions
  name: 'II.1 On-Road Transport Emissions'
  part_of: transport_emissions
- id: railways_emissions
  name: 'II.2 Railways Emissions'
  part_of: transport_emissions
- id